# Importe as bibliotecas necessárias

In [ ]:

# Instalando as bibliotecas necessárias
%pip install -U pip setuptools wheel
%pip install pymongo
%pip install pandas
%pip install nltk
%pip install wordcloud
%pip install spacy
%pip install matplotlib
%pip install numpy==1.26.4
%pip install -U scikit-learn
%pip install unidecode

In [2]:
# Importando as bibliotecas
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from wordcloud.wordcloud import STOPWORDS
from spacy.lang.pt.stop_words import STOP_WORDS
import re
import string
from datetime import datetime, timedelta
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import unidecode
from db import connection_db as conndb
from db import filters

In [3]:
mongo_connection = conndb.MongoDBConnection(uri='mongodb://localhost:27017/', database_name='dadosVivamente', collection_name='dadosSemFiltros')
mongo_connection.connect()
db = mongo_connection.db
collection = db.collection

Conexão estabelecida com sucesso ao banco de dados.


In [4]:
collection_filters = filters.CollectionFilters(db, 'dadosSemFiltros', 'dadosComFiltrosIniciais')
filtered_collection = collection_filters.apply_pipeline1()

Coleção: dadosSemFiltros
A coleção já existe


In [ ]:
# Pegar apenas os documentos que possuem a data de criação do primeiro post com pelo 7 anos de diferença da data de criação do último post
if 'posts7anos2anos' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['posts7anos2anos']
else:
    # 7 anos em milissegundos
    sete_anos_millis = 7 * 365.25 * 24 * 60 * 60 * 1000

    # 2 anos em milissegundos
    dois_anos_millis = 2 * 365.25 * 24 * 60 * 60 * 1000

    pipeline2 = [
        {
            '$addFields': {
                'firstPostDate': {
                    '$toLong': {
                        '$arrayElemAt': [
                            '$posts.created_time', -1
                        ]
                    }
                },
                'lastPostDate': {
                    '$toLong': {
                        '$arrayElemAt': [
                            '$posts.created_time', 0
                        ]
                    }
                }
            }
        },
        {
            '$addFields': {
                'diff': {
                    '$subtract': [
                        '$lastPostDate', '$firstPostDate'
                    ]
                }
            }
        },
        {
            '$match': {
                '$and': [
                    {'diff': {'$gte': dois_anos_millis}},
                    {'diff': {'$lte': sete_anos_millis}}
                ]
            }
        },
        {
            '$out': 'posts7anos2anos'
        }
    ]
    collection.aggregate(pipeline2)
    collection = db['posts7anos2anos']

In [ ]:
if 'postsComBDIAndInfos' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['postsComBDIAndInfos']
else:
    pipeline3 = [
        {
            '$unwind': '$posts'
        }, {
            '$project': {
                '_id': '$posts._id',
                'idade': 1,
                'sexo': 1,
                'id_usuario': 1,
                'nivel': {
                    '$let': {
                        'vars': {
                            'dividedValue': {
                                '$divide': [
                                    {
                                        '$arrayElemAt': [
                                            '$respostas.nivel', 0
                                        ]
                                    }, 21
                                ]
                            }
                        },
                        'in': {
                            '$cond': {
                                'if': {
                                    '$gte': [
                                        {
                                            '$subtract': [
                                                '$$dividedValue', {
                                                    '$trunc': '$$dividedValue'
                                                }
                                            ]
                                        }, 0.5
                                    ]
                                },
                                'then': {
                                    '$add': [
                                        {
                                            '$trunc': '$$dividedValue'
                                        }, 1
                                    ]
                                },
                                'else': {
                                    '$trunc': '$$dividedValue'
                                }
                            }
                        }
                    }
                },
                'pessimismo': {
                    '$arrayElemAt': [
                        '$respostas.pessimismo', 0
                    ]
                },
                'tristeza': {
                    '$arrayElemAt': [
                        '$respostas.tristeza', 0
                    ]
                },
                'fracasso': {
                    '$arrayElemAt': [
                        '$respostas.fracasso', 0
                    ]
                },
                'prazer': {
                    '$arrayElemAt': [
                        '$respostas.prazer', 0
                    ]
                },
                'culpa': {
                    '$arrayElemAt': [
                        '$respostas.culpa', 0
                    ]
                },
                'punicao': {
                    '$arrayElemAt': [
                        '$respostas.punicao', 0
                    ]
                },
                'estima': {
                    '$arrayElemAt': [
                        '$respostas.estima', 0
                    ]
                },
                'critica': {
                    '$arrayElemAt': [
                        '$respostas.critica', 0
                    ]
                },
                'suicida': {
                    '$arrayElemAt': [
                        '$respostas.suicida', 0
                    ]
                },
                'choro': {
                    '$arrayElemAt': [
                        '$respostas.choro', 0
                    ]
                },
                'agitacao': {
                    '$arrayElemAt': [
                        '$respostas.agitacao', 0
                    ]
                },
                'interesse': {
                    '$arrayElemAt': [
                        '$respostas.interesse', 0
                    ]
                },
                'indecisao': {
                    '$arrayElemAt': [
                        '$respostas.indecisao', 0
                    ]
                },
                'desvalorizacao': {
                    '$arrayElemAt': [
                        '$respostas.desvalorizacao', 0
                    ]
                },
                'energia': {
                    '$arrayElemAt': [
                        '$respostas.energia', 0
                    ]
                },
                'sono': {
                    '$arrayElemAt': [
                        '$respostas.sono', 0
                    ]
                },
                'irritabilidade': {
                    '$arrayElemAt': [
                        '$respostas.irritabilidade', 0
                    ]
                },
                'apetite': {
                    '$arrayElemAt': [
                        '$respostas.apetite', 0
                    ]
                },
                'concentracao': {
                    '$arrayElemAt': [
                        '$respostas.concentracao', 0
                    ]
                },
                'fadiga': {
                    '$arrayElemAt': [
                        '$respostas.fadiga', 0
                    ]
                },
                'int_sexo': {
                    '$arrayElemAt': [
                        '$respostas.int_sexo', 0
                    ]
                },
                'quantAmigos': '$friends.summary.total_count',
                'postMessage': '$posts.message',
                'postStory': '$posts.story',
                'postCreated_time': '$posts.created_time',
                'diaDaSemana': {
                    '$switch': {
                        'branches': [
                            {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 1
                                    ]
                                },
                                'then': 'Domingo'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 2
                                    ]
                                },
                                'then': 'Segunda-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 3
                                    ]
                                },
                                'then': 'Terça-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 4
                                    ]
                                },
                                'then': 'Quarta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 5
                                    ]
                                },
                                'then': 'Quinta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 6
                                    ]
                                },
                                'then': 'Sexta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 7
                                    ]
                                },
                                'then': 'Sábado'
                            }
                        ],
                        'default': 'Desconhecido'
                    }
                },
                'hora': {
                    '$hour': '$posts.created_time'
                },
                'minutos': {
                    '$minute': '$posts.created_time'
                },
                'diaDoMes': {
                    '$dayOfMonth': '$posts.created_time'
                },
                'mes': {
                    '$month': '$posts.created_time'
                },
                'ano': {
                    '$year': '$posts.created_time'
                }
            }
        }, {
            '$out': 'postsComBDIAndInfos'
        }
    ]
    collection.aggregate(pipeline3)
    collection = db['postsComBDIAndInfos']

In [ ]:
# Verificando se a coleção existe
if 'postsComBDIAndInfosFiltroDataPosts' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['postsComBDIAndInfosFiltroDataPosts']
else:
    # Primeira parte: Filtrar documentos com 'diaDaSemana' == 'Desconhecido'
    pipeline4 = [
        {
            '$match': {
                'diaDaSemana': {'$ne': 'Desconhecido'}
            }
        },
        {
            '$out': 'postsComBDIAndInfosFiltroDataPosts'
        }
    ]
    collection.aggregate(pipeline4)
    collection = db['postsComBDIAndInfosFiltroDataPosts']

In [ ]:
pipeline5 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            }
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_unicos'  # Conta quantos grupos únicos foram formados
    }
]

# Executando a agregação
resultado = list(collection.aggregate(pipeline5))

# Obtendo o resultado
if resultado:
    print(f"Usuários únicos: {resultado[0]['numero_de_usuarios_unicos']}")
else:
    print("Nenhum usuário encontrado.")

In [ ]:
# Contando quantos usuarios são do sexo feminino e quantos são do sexo masculino
pipeline6 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            },
            'sexo': 'F'
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_femininos'
    }
]
quant_feminino = list(collection.aggregate(pipeline6))
print(f"Quantidade de Usuarios Femininos: {quant_feminino[0]['numero_de_usuarios_femininos']}")

pipeline7 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            },
            'sexo': 'M'
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_masculinos'
    }
]
quant_masculino = list(collection.aggregate(pipeline7))
print(f"Quantidade de Usuarios Masculinos: {quant_masculino[0]['numero_de_usuarios_masculinos']}")

# Filtro de 6 meses antes do mês da coleta

In [ ]:
if 'posts6Meses' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['posts6Meses']
else:
    data_inicio = datetime(2017, 12, 1)

    data_fim = data_inicio - timedelta(days=6 * 30)

    pipeline8 = [
        {
            '$match': {
                'postCreated_time': {
                    '$gte': data_fim,
                    '$lt': data_inicio
                }
            }
        },
        {
            '$out': 'posts6Meses'
        }
    ]
    collection.aggregate(pipeline8)
    collection = db['posts6Meses']

In [ ]:
# Filtrando documentos com o atributo maior que 3
filtro = {"suicida": {"$eq": "3"}}
documentos = collection.find(filtro)

In [ ]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [ ]:
df['data'] = pd.to_datetime(df['postCreated_time'])
df.head()

In [ ]:
print(df.dtypes)

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [ ]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv('dados/com_filtros_datas/6meses/so_suicida_6_meses.csv', index=False)

In [ ]:
# filtrar pelo id_usuario
# df = df[df['id_usuario'] == '1022864967872047']

In [ ]:
# Agrupar por usuário, mês e ano
posts_grouped = df.groupby(['id_usuario', 'mes', 'ano']).size().reset_index(name='quantidade')

# Adicionar coluna com o período
posts_grouped['periodo'] = posts_grouped['mes'].astype(str) + '/' + posts_grouped['ano'].astype(str)

# Plotar quantidade de posts por usuário

In [ ]:
# # plotar quantidade de posts por usuario
# 
# # Criar o gráfico de linha para cada usuário
# for usuario in posts_grouped['id_usuario'].unique():
#     df_usuario = posts_grouped[posts_grouped['id_usuario'] == usuario].copy()
# 
#     # Convertendo a coluna 'periodo' para datetime para garantir a ordenação correta
#     df_usuario['periodo'] = pd.to_datetime(df_usuario['periodo'], format='%m/%Y', errors='coerce')
# 
#     # Ordenar os dados por 'periodo'
#     df_usuario = df_usuario.sort_values('periodo')
# 
#     # Configurar o gráfico de linha
#     plt.figure(figsize=(20, 8))  # Aumentar o tamanho da figura
# 
#     plt.plot(df_usuario['periodo'].dt.strftime('%m/%Y'), df_usuario['quantidade'], marker='o', linestyle='-',
#              color='blue')
# 
#     # Adicionar título e rótulos
#     plt.title(f'Quantidade de Posts por Mês/Ano - Usuário: {usuario}')
#     plt.xlabel('Mês/Ano')
#     plt.ylabel('Quantidade de Posts')
# 
#     # Melhorar a legibilidade dos rótulos do eixo X
#     plt.xticks(rotation=45, ha='right', fontsize=10)
# 
#     # Aumentar o pad dos rótulos do eixo X
#     plt.gca().tick_params(axis='x', pad=14)  # Aumenta o espaço entre os rótulos e o eixo
# 
#     # Adicionar grid
#     plt.grid(True, axis='y')
# 
#     # Ajustar o layout para evitar sobreposição, com mais padding
#     plt.tight_layout(pad=8.0)  # Aumentar o padding geral do layout
# 
#     # Adicionar espaço extra ao layout se necessário
#     plt.subplots_adjust(bottom=0.2)  # Adiciona mais espaço abaixo dos rótulos do eixo X
# 
#     # Salvando o gráfico no diretório dados/com_filtros_datas/6meses/graficos
#     plt.savefig(f'dados/com_filtros_datas/6meses/graficos/quantidade_posts_{usuario}.png')
# 
#     # Mostrar o gráfico
#     plt.show()
# 
#     # Fechar a figura explicitamente para liberar memória
#     plt.close()

# Efentuando a limpeza dos dados

In [ ]:
def get_stopwords() -> list:
    portuguese_ingles_stopwords = []

    # Stopwords em português
    portugues = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in
                 nltk.corpus.stopwords.words('portuguese')]
    portuguese_ingles_stopwords.extend(portugues)

    # Stopwords em inglês
    ingles = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in
              nltk.corpus.stopwords.words('english')]
    portuguese_ingles_stopwords.extend(ingles)

    # Stopwords de diferentes fontes
    stopwords_wordcloud = [unidecode.unidecode(palavra.lower().replace(" ", "")) for palavra in STOPWORDS]
    portuguese_ingles_stopwords.extend(stopwords_wordcloud)

    stopwords_sklearn = [unidecode.unidecode(word.lower().replace(" ", "")) for word in STOP_WORDS]
    portuguese_ingles_stopwords.extend(stopwords_sklearn)

    # Adicionar stopwords do arquivo customizado
    with open('dados/datasets/stopwords-pt.txt', 'r', encoding='utf-8') as words:
        custom_stopwords = [unidecode.unidecode(word.lower().strip()) for word in words if word.strip()]
    portuguese_ingles_stopwords.extend(custom_stopwords)

    # Adicionar nomes
    with open('dados/datasets/nomes.txt', 'r', encoding='utf-8') as words:
        nomes_stopwords = [unidecode.unidecode(word.lower().strip()) for word in words if word.strip()]
    portuguese_ingles_stopwords.extend(nomes_stopwords)

    # Remover duplicatas e ordenar a lista
    portuguese_ingles_stopwords = list(set(portuguese_ingles_stopwords))
    portuguese_ingles_stopwords.sort()

    return portuguese_ingles_stopwords


def remocao_stopword(string, lista_stopwords) -> str:
    a = [i for i in string.split() if i not in lista_stopwords]
    return ' '.join(a)


def remove_caracteres(text) -> str:
    text = text.lower().strip()
    text = re.compile(r'<.*?>').sub('', text)
    text = re.compile(r'[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r"\$", "", text)
    text = re.sub(r"https?:\/\/.*[\r\n]*", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r'https?:\/\/[\r\n],"[\r\n]"', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'[^a-zà-ù ]', ' ', text)
    text = re.sub(r'k{2,}', '', text, flags=re.IGNORECASE)
    text = re.sub(r'j{2,}', '', text, flags=re.IGNORECASE)
    text = re.compile(r"["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", flags=re.UNICODE).sub(r'', text)
    return text


def obter_pos_tag(token) -> str:
    if token.startswith('J'):
        return wordnet.ADJ
    elif token.startswith('V'):
        return wordnet.VERB
    elif token.startswith('N'):
        return wordnet.NOUN
    elif token.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def lematizacao(string) -> str:
    token = word_tokenize(string)
    word_pos_tags = nltk.pos_tag(token)
    wl = WordNetLemmatizer()
    a = [wl.lemmatize(tag[0], obter_pos_tag(tag[1])) for idx, tag in
         enumerate(word_pos_tags)]
    return " ".join(a)


def preprocessamento_texto(texto_limpo) -> str:
    lista_stopwords = get_stopwords()
    texto_limpo = remove_caracteres(texto_limpo)
    texto_limpo = remocao_stopword(texto_limpo, lista_stopwords)
    texto_limpo = lematizacao(texto_limpo)
    return texto_limpo

In [ ]:
df['postMessageLimpo'] = df['postMessage'].fillna('').apply(preprocessamento_texto)
df.head()
corpus = df['postMessageLimpo'].tolist()

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
# Carregar stopwords em português para os TF-IDF e Bag of Words
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')

In [ ]:
# Criar o vetorizador TF-IDF com parâmetros ajustados
vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    max_features=None,
    max_df=0.40,
    min_df=5,
    ngram_range=(1, 1),
    sublinear_tf=True
)

corpus_tfidf = df['postMessageLimpo'].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus_tfidf)

# Obter as palavras
palavras_tfidf = vectorizer.get_feature_names_out()
# Lista para armazenar os resultados
resultados_tfidf = []

# Iterar sobre cada usuário
for usuario in df['id_usuario'].unique():
    # Filtrar os textos do usuário
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        # Extrair as palavras com maior score TF-IDF para o usuário
        user_tfidf = tfidf_matrix[indices_usuario]
        user_tfidf_mean = np.asarray(user_tfidf.mean(axis=0)).flatten()

        # Obter os índices das 10 palavras com maior score
        top_10_indices = user_tfidf_mean.argsort()[-10:][::-1]

        # Adicionar as palavras e seus scores à lista de resultados
        for index in top_10_indices:
            resultados_tfidf.append({
                'id_usuario': usuario,
                'palavra': palavras_tfidf[index],
                'score': user_tfidf_mean[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_tfidf = pd.DataFrame(resultados_tfidf)
resultados_df_tfidf.to_csv('dados/com_filtros_datas/6meses/so_suicida_resultados_tfidf_unigramas.csv', index=False)

print(resultados_df_tfidf)

Usando o algoritmo Bag of Words

In [ ]:
# Criar o vetorizador Bag of Words com parâmetros ajustados
vectorizer = CountVectorizer(
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_df=0.85,
    min_df=5
)

corpus_bow = df['postMessageLimpo'].tolist()
bow_matrix = vectorizer.fit_transform(corpus_bow)

palavras_bow = vectorizer.get_feature_names_out()

resultados_bow = []

for usuario in df['id_usuario'].unique():
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        user_bow = bow_matrix[indices_usuario]
        user_bow_sum = np.asarray(user_bow.sum(axis=0)).flatten()

        top_10_indices_bow = user_bow_sum.argsort()[-10:][::-1]

        for index in top_10_indices_bow:
            resultados_bow.append({
                'id_usuario': usuario,
                'palavra': palavras_bow[index],
                'contagem': user_bow_sum[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_bow = pd.DataFrame(resultados_bow)
resultados_df_bow.to_csv('dados/com_filtros_datas/6meses/so_suicida_resultados_bow_unigramas.csv', index=False)

print(resultados_df_bow)